In [39]:
from dotenv import load_dotenv

load_dotenv()

True

In [40]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

llm = ChatOpenAI(model="gpt-3.5-turbo")

messages = [
    HumanMessage(content="List 8 wonders of the word")
]

result = llm.invoke(messages)

wonders_list = result.content.split("\n")

for wonder in wonders_list:
    print(wonder)

1. Great Wall of China
2. Petra, Jordan
3. Christ the Redeemer, Brazil
4. Machu Picchu, Peru
5. Chichen Itza, Mexico
6. Colosseum, Italy
7. Taj Mahal, India
8. Great Pyramid of Giza, Egypt


## Streaming and Batching


In [41]:
for chunk in llm.stream("8 Wonders of World"):
    print(chunk.content)


1
.
 Great
 Wall
 of
 China


2
.
 Petra
,
 Jordan


3
.
 Christ
 the
 Rede
emer
 statue
 in
 Rio
 de
 Janeiro
,
 Brazil


4
.
 Mach
u
 Pic
chu
,
 Peru


5
.
 Taj
 Mah
al
,
 India


6
.
 Col
os
se
um
 in
 Rome
,
 Italy


7
.
 Ch
ichen
 It
za
,
 Mexico


8
.
 Pyramid
 of
 G
iza
,
 Egypt



In [42]:
result = llm.batch(
    [
        "What's 2 +2 ?",
        "What's 34.9 + 23.78?"
    ]
)
for _result in result:
    print(_result.content)

2 + 2 = 4
58.68


## System & Human Message

In [67]:
from langchain.schema import SystemMessage

messages = [
    SystemMessage(content="You are Virat Kohli"),
    HumanMessage(content="Which crickrt bat manufacturer you are associated with?")
]

result = llm.invoke(messages)
result.content

'I am associated with the cricket bat manufacturer, MRF.'

## Prompt Template

In [44]:
from langchain.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "tell me a {objective} joke about {content}"
)

filled_prompt = prompt_template.format(objective="funny", content="robots")

response = llm.invoke(filled_prompt)
response.content

'Why did the robot go on a diet?\n\nBecause he had a byte problem!'

## Chat Prompt Template

In [45]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Defining a chat template with various roles
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful AI Mathematics teacher bot. Your name is {name}"),
        HumanMessage(content="Hello, How are you doing?"),
        AIMessage(content="I'm doing well, thanks!"),
        HumanMessage(content="{math_problem}")
    ]
)

math_problem = input("Enter Math problem:")

formatted_messages = chat_template.format_messages(name="Bob", math_problem=math_problem)

for message in formatted_messages:
    print(message.content)

You are a helpful AI Mathematics teacher bot. Your name is {name}
Hello, How are you doing?
I'm doing well, thanks!
{math_problem}


## Output Parsers

### Simple Json Parser

In [46]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers.json import SimpleJsonOutputParser

json_parser = SimpleJsonOutputParser()

template = """
Answer the user's question:
{question}
Return the answer as a JSON object with keys 'birth_place' and 'birth_date'.
"""

json_prompt = PromptTemplate.from_template(template)

json_chain = json_prompt | llm | json_parser

input_data = {"question": "When and where were Elon Musk and Bill Gates born?"}

output = json_chain.invoke(input_data)
output

{'birth_place': {'Elon Musk': 'Pretoria, South Africa',
  'Bill Gates': 'Seattle, Washington, United States'},
 'birth_date': {'Elon Musk': 'June 28, 1971',
  'Bill Gates': 'October 28, 1955'}}

### CSV Parser

In [47]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

template = """
Answer the user's question:
{question}
Return the answer as a comma-separated list with the format: "Country, Capital, Independence Date".
Only include countries in Asia.
"""

csv_prompt = PromptTemplate.from_template(template)

csv_chain = csv_prompt | llm | csv_parser

input_data = {"question": "Give details of countries in Aisa"}

output = csv_chain.invoke(input_data)
output

['Afghanistan',
 'Kabul',
 '1919\nBahrain',
 'Manama',
 '1971\nBangladesh',
 'Dhaka',
 '1971\nBhutan',
 'Thimphu',
 '1949\nBrunei',
 'Bandar Seri Begawan',
 '1984\nCambodia',
 'Phnom Penh',
 '1953\nChina',
 'Beijing',
 '1949\nIndia',
 'New Delhi',
 '1947\nIndonesia',
 'Jakarta',
 '1945\nIran',
 'Tehran',
 '1935\nIraq',
 'Baghdad',
 '1932\nIsrael',
 'Jerusalem',
 '1948\nJapan',
 'Tokyo',
 '660 BC\nJordan',
 'Amman',
 '1946\nKazakhstan',
 'Nur-Sultan',
 '1991\nKuwait',
 'Kuwait City',
 '1961\nKyrgyzstan',
 'Bishkek',
 '1991\nLaos',
 'Vientiane',
 '1949\nLebanon',
 'Beirut',
 '1943\nMalaysia',
 'Kuala Lumpur',
 '1957\nMaldives',
 'Malé',
 '1965\nMongolia',
 'Ulaanbaatar',
 '1921\nMyanmar',
 'Naypyidaw',
 '1948\nNepal',
 'Kathmandu',
 '1768\nNorth Korea',
 'Pyongyang',
 '1948\nOman',
 'Muscat',
 '1650\nPakistan',
 'Islamabad',
 '1947\nPalestine',
 'East Jerusalem',
 '1988\nPhilippines',
 'Manila',
 '1898\nQatar',
 'Doha',
 '1971\nSaudi Arabia',
 'Riyadh',
 '1932\nSingapore',
 'Singapore',
